In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())
import shutil
import numpy as np
import tensorflow as tf
from yolov3.dataset import Dataset

from yolov3.yolov4 import Create_Yolo,Create_Yolo1,compute_loss
from yolov3.yolov4 import compute_loss1
from yolov3.utils import load_yolo_weights
from yolov3.utils import load_yolo_weights, detect_image, image_preprocess, postprocess_boxes, nms, read_class_names

from yolov3.configs import *
from evaluate_mAP import get_mAP
    
from supportFunctions import draw_bbox

from yolov3.dataset import Dataset

trainset = Dataset('train')
testset = Dataset('test')



[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 11979077096272644227
]


In [2]:
def read_class_names(class_file_name):
    # loads class name from a file
    names = {}
    with open(class_file_name, 'r') as data:
        for ID, name in enumerate(data):
            names[ID] = name.strip('\n')
    return names
NUM_CLASS = len(read_class_names(TRAIN_CLASSES))
NUM_CLASS

21

In [2]:
import os

os.getcwd()

'/home/aniketag/Documents/phd/TensorFlow-2.x-YOLOv3_simula/YOLOPhosc'

In [3]:
#from yolov3.yolov4 import Create_YoloPhosc
#from yolov3.yolov3 import Create_Yolov3
global TRAIN_FROM_CHECKPOINT


from yolov3.yolov3 import *
from yoloModify import yolo33
yoloNew=yolo33(input_size=416, channels=3, training=True, CLASSES=TRAIN_CLASSES)

gpus = tf.config.experimental.list_physical_devices('GPU')
#print(f'GPUs {gpus}')

if len(gpus) > 0:
    try: tf.config.experimental.set_memory_growth(gpus[0], True)
    except RuntimeError: pass

if os.path.exists(TRAIN_LOGDIR): shutil.rmtree(TRAIN_LOGDIR)
writer = tf.summary.create_file_writer(TRAIN_LOGDIR)

if len(gpus) > 0:
    try: tf.config.experimental.set_memory_growth(gpus[0], True)
    except RuntimeError: pass

if os.path.exists(TRAIN_LOGDIR): shutil.rmtree(TRAIN_LOGDIR)
writer = tf.summary.create_file_writer(TRAIN_LOGDIR)

steps_per_epoch = len(trainset)
global_steps = tf.Variable(1, trainable=False, dtype=tf.int64)
warmup_steps = TRAIN_WARMUP_EPOCHS * steps_per_epoch
total_steps = TRAIN_EPOCHS * steps_per_epoch

if YOLO_TYPE == "yolov3":
    Darknet_weights = YOLO_V3_TINY_WEIGHTS if TRAIN_YOLO_TINY else YOLO_V3_WEIGHTS

TRAIN_TRANSFER=False

if TRAIN_TRANSFER:
    
    """
        https://fixexception.com/tensorflow/the-first-argument-to-layer-call-must-always-be-passed/
    """
    from tensorflow.keras.layers import Input
    input_layer = Input([416, 416, 3])
    
    print(" 1.trying to load")
    Darknet =yoloNew(input_layer)
    
    #Darknet = Create_Yolo(input_size=YOLO_INPUT_SIZE, CLASSES=YOLO_COCO_CLASSES)
    #Darknet = yoloNew(input_size=YOLO_INPUT_SIZE, CLASSES=YOLO_COCO_CLASSES)
    #print("Darknet_weights=",os.path.isfile(Darknet_weights))
    
    print(" 2.trying to load")

    
    load_yolo_weights(Darknet, Darknet_weights) # use darknet weights

    print(" 2.trying to load")

    
    
#yolo = Create_Yolo(input_size=YOLO_INPUT_SIZE, training=True, CLASSES=TRAIN_CLASSES)

#yolo=Create_Yolov3(input_size=YOLO_INPUT_SIZE, training=True, CLASSES=TRAIN_CLASSES)

#TRAIN_MODEL_NAME=

TRAIN_FROM_CHECKPOINT=True

if TRAIN_FROM_CHECKPOINT:
    try:
        print("\n\t load checkpoint!!!")
        yoloNew.load_weights(f"./checkpoints/{TRAIN_MODEL_NAME}")
        print("\n\t loading complete")

    except ValueError:
        print("Shapes are incompatible, transfering Darknet weights")
        TRAIN_FROM_CHECKPOINT = False

'''
if TRAIN_TRANSFER and not TRAIN_FROM_CHECKPOINT:
    for i, l in enumerate(Darknet.layers):
        layer_weights = l.get_weights()
        if layer_weights != []:
            try:
                yolo.layers[i].set_weights(layer_weights)
            except:
                print("skipping", yolo.layers[i].name)
'''

optimizer = tf.keras.optimizers.Adam()

#mAP_model = Create_Yolo(input_size=YOLO_INPUT_SIZE, CLASSES=TRAIN_CLASSES) # create second model to measure mAP
#best_val_loss = 1000 # should be large at start

#print("\n\t mAP_model =",mAP_model)

#print("\n\t yolo:",yolo.summary())





**

	 load checkpoint!!!

	 loading complete


In [4]:
def validate_step(image_data, target):
    with tf.GradientTape() as tape:
        pred_result = yolo(image_data, training=False)
        
        print("\n\t 1.pred_result validate:",len(pred_result[0]))
        pred_result =pred_result[0]
        print("\n\t 2.pred_result validate:",pred_result.shape)

#         if type(pred_result) is list:
#             pred_result =pred_result[0]
        giou_loss=conf_loss=prob_loss=0

        # optimizing process
        grid = 1 if not TRAIN_YOLO_TINY else 2
        for i in range(grid):
            conv, pred = pred_result[i*2], pred_result[i*2+1]
            loss_items = compute_loss1(pred, conv, *target[i], i, CLASSES=TRAIN_CLASSES)
            giou_loss += loss_items[0]
            conf_loss += loss_items[1]
            prob_loss += loss_items[2]

        total_loss = giou_loss + conf_loss + prob_loss

    return giou_loss.numpy(), conf_loss.numpy(), prob_loss.numpy(), total_loss.numpy()

phoscLoss=0
NUM_CLASS = len(read_class_names(TRAIN_CLASSES))

def train_step(nm,counter,image_data, target):
    #print("\n\t inside train")
    with tf.GradientTape() as tape:
        #pred_result = yolo(image_data, training=True)
        pred_result = yoloNew(image_data, training=True)
        
        
        """
        print("\n\t image_data in train:",image_data.shape)
        print("\n\t in train pred_result len=",len(pred_result))
        print("\n\t in train pred_result 0=",pred_result[0].shape)
        print("\n\t in train pred_result 1=",pred_result[1].shape)
        print("\n\t in train pred_result 0=",pred_result[2].shape)

        print("\n\t in train pred_result 0=",pred_result[3].shape)
        print("\n\t in train pred_result 1=",pred_result[4].shape)
        print("\n\t in train pred_result 0=",pred_result[5].shape)
        """
        
        """
             image_data in train: (1, 416, 416, 3)
             in train pred_result len= 6
             in train pred_result 0= (1, 52, 52, 2385)
             in train pred_result 1= (1, 52, 52, 3, 795)
             in train pred_result 0= (1, 26, 26, 2385)
             in train pred_result 0= (1, 26, 26, 3, 795)
             in train pred_result 1= (1, 13, 13, 2385)
             in train pred_result 0= (1, 13, 13, 3, 795)        
        """
        
        
        
        
        #print("\n\t in train pred_result 1=",pred_result[3].shape)
        
    
        image2=tf.squeeze(image_data)
        """
        try:
            print("image2=",image2.shape)
        except Exception as e:
            print("image2=",image2.size)
        """
            
        """
            image2= (416, 416, 3)
        """
        
        
        giou_loss=conf_loss=prob_loss=phocLoss1=phosLoss2=0

        # optimizing process
        grid = 3 if not TRAIN_YOLO_TINY else 2

        for i in range(grid):
            conv, pred = pred_result[i*2], pred_result[i*2+1]
            #print("\n\t conv train:",conv.shape,"\t i:",i)
            #print("\n\t pred train:",pred.shape,"\t i:",i)
            
            loss_items = compute_loss1(pred, conv, *target[i], i, CLASSES=TRAIN_CLASSES)
            giou_loss += loss_items[0]
            conf_loss += loss_items[1]
            prob_loss += loss_items[2]
            phocLoss1+=loss_items[3]
            phosLoss2+=loss_items[4]

            
        total_loss = giou_loss + conf_loss + prob_loss
        total_loss1=phoscLoss+giou_loss + conf_loss + prob_loss+phocLoss1+phosLoss2
        #total_loss1=phoscLoss+total_loss
        #print("\n\t phoscLoss1:",phoscLoss1,"\t phoscLoss2:",phoscLoss2)

        #print("\n\t total_loss =",total_loss,"\ttotal_loss1 =",total_loss1)
        
        if 0:#total_loss<40: 
            print("\n\t localization+PHOS loss considered ")
            
            total_loss1=0.2*(giou_loss + conf_loss + prob_loss)+(phocLoss1+phosLoss2+phoscLoss)
            gradients = tape.gradient(total_loss1, yoloNew.trainable_variables)
        else:
            #print("\n\t\t only total loss considered ",total_loss.numpy())

            gradients = tape.gradient(total_loss, yoloNew.trainable_variables)
            
        optimizer.apply_gradients(zip(gradients, yoloNew.trainable_variables))

        global_steps.assign_add(1)
        if global_steps < warmup_steps:# and not TRAIN_TRANSFER:
            lr = global_steps / warmup_steps * TRAIN_LR_INIT
        else:
            lr = TRAIN_LR_END + 0.5 * (TRAIN_LR_INIT - TRAIN_LR_END)*(
                (1 + tf.cos((global_steps - warmup_steps) / (total_steps - warmup_steps) * np.pi)))
        optimizer.lr.assign(lr.numpy())

        # writing summary data
        with writer.as_default():
            tf.summary.scalar("lr", optimizer.lr, step=global_steps)
            tf.summary.scalar("total_loss", total_loss, step=global_steps)
            tf.summary.scalar("total_loss1", total_loss1, step=global_steps)

            tf.summary.scalar("giou_loss", giou_loss, step=global_steps)
            tf.summary.scalar("conf_loss", conf_loss, step=global_steps)
            tf.summary.scalar("prob_loss", prob_loss, step=global_steps)

            tf.summary.scalar("phocLoss1", phocLoss1, step=global_steps)
            tf.summary.scalar("phosLoss2",phosLoss2, step=global_steps)
            tf.summary.scalar("phocLoss1+phosLoss2",(phocLoss1+phosLoss2), step=global_steps)

        writer.flush()
    #print("\n\t total_loss =",total_loss.numpy(),"\t total_loss1 =",total_loss1.numpy())
    #print("\n\t phocLoss1=",phocLoss1.numpy(),"\t phosLoss2:",phosLoss2.numpy())

    
    if counter%100==0:
    
        pred_result = yoloNew(image_data, training=False)
        #print("\n\t image_data pred:",image_data.shape)
        #print("\n\t len(pred_result) pred:",len(pred_result))
        image2=tf.squeeze(image_data)

        #print("\n\t\t 1.pred_result pred=",pred_result[0].shape)
        #print("\n\t\t 2.pred_result pred=",pred_result[1].shape)
        #print("\n\t\t 3.pred_result pred=",pred_result[2].shape)

        #print("\n\t image2=",image2.shape)
        #pred_bbox = [tf.reshape(x, (-1, tf.shape(x)[-1])) for x in pred_result]
        #print("\n\t pred_bbox =",pred_bbox )
        #pred_bbox = tf.concat(pred_bbox, axis=0)


        # optimizing process
        grid = 3 if not TRAIN_YOLO_TINY else 2
        for i in range(grid):
            """
            if i>0:
                continue
            """
            '''
                these r 3 feature maps of size 52*52,26*26,13*13,

            '''
            conv, pred = pred_result[i*2], pred_result[i*2+1]
            #print("\n\t\t conv pred:",conv.shape)
            #print("\n\t\t pred pred=",pred.shape)


            ''' 
                (xc,yc,w,h,obj,pro) is last dimension of size 6

                pred = (1, 13, 13, 3, 6) is reshaped to 
                pred_bbox = (507, 6)   169 is len of single anchor tot 3 vectors             
                 #2.pred_result: (1, 52, 52, 2325)
                conv: (1, 26, 26, 18)
                pred = (1, 26, 26, 3, 6)
                pred_bbox = (2028, 6)  676 is lenght of single anchor tot 3 vectors

                image_data= (1, 416, 416, 3)
                pred = (1, 52, 52, 3, 6)
                pred_bbox = (8112, 6)  2704 is len of single anchor tot 3 vectors

            '''

            pred_bbox = [tf.reshape(x, (-1, tf.shape(x)[-1])) for x in pred]
            #print("\n\t 1.pred_bbox pred=",len(pred_bbox))

            pred_bbox = tf.concat(pred_bbox, axis=0)

            #print("\n\t 2.pred_bbox pred=",pred_bbox.shape)

            ''' 
            this removes invalid boxes like boxes having low confidence or 
            which has size greater than image

            '''
            #print("\n\t pred_bbox=",len(pred_bbox))
            bboxes = postprocess_boxes(pred_bbox, image2.numpy(), YOLO_INPUT_SIZE,0.25,NUM_CLASS)
            #print("\n\t befor nms bboxes =",bboxes.shape)


            bboxes = nms(bboxes, 0.1, method='nms')


            #print("\n\t after nms 1.bboxes =",len(bboxes))
            #print("\n\t after nms 2.bboxes =",bboxes)

            #draw_bbox() missing 1 required positional argument: 'bboxes
            '''
                 draw_bbox(nm,counter,level, image, bboxes, CLASSES=YOLO_COCO_CLASSES, show_label=True, show_confidence=True,
                  Text_colors=(255, 255, 0), rectangle_colors='', tracking=False)
            '''
            #print("\n\t bboxes len:",len(bboxes))


            if 1:
                try:
                    if bboxes:
                    #print("image2.numpy()",image2.numpy().shape)
                        resultImage = draw_bbox(nm,counter,i,255*image2.numpy(),bboxes,None,CLASSES=TRAIN_CLASSES, rectangle_colors="")
                except Exception as e:
                    pass
            #cv2.imwrite(f'./pred/{counter}.png',resultImage)
            #input("check")


    return global_steps.numpy(), optimizer.lr.numpy(), giou_loss.numpy(), conf_loss.numpy(), prob_loss.numpy(), total_loss.numpy(),total_loss1.numpy()

#mAP_model = Create_Yolo(input_size=YOLO_INPUT_SIZE, CLASSES=TRAIN_CLASSES) # create second model to measure mAP


In [ ]:
best_val_loss = 1000
from supportFunctions import draw_bbox

minLoss=100000.0
for epoch in range(5000):
    totLoss=0
    #print("\n\t epoch:",epoch,"\tminLoss:",minLoss," totLoss:",totLoss)
    #counter,(image_data, target,allImagesName)
    for counter,(image_data, target,allImagesName) in enumerate(trainset):
        
        if counter%100==0:
            print(" counter=",counter)
        
        nm=str(np.random.randint(10000))+".png"
        nm=str(counter)+".png"

        results = train_step(nm,counter,image_data, target)

        #pred_result = yolo(image_data, training=True)
        
        #pred_result = yoloNew(image_data, training=True)
        
        cur_step = results[0]%steps_per_epoch
        totLoss+=results[-2]
            
    print("\n\t epoch:",epoch,"\tminLoss:",minLoss," totLoss:",totLoss)
         
    if totLoss<minLoss:#TRAIN_SAVE_CHECKPOINT and TRAIN_SAVE_BEST_ONLY:
        #save_directory = os.path.join(TRAIN_CHECKPOINTS_FOLDER, TRAIN_MODEL_NAME+"_total_loss2_{:7.2f}".format(results[-1]))
        save_directory = os.path.join(TRAIN_CHECKPOINTS_FOLDER, TRAIN_MODEL_NAME)

        yoloNew.save_weights(save_directory)
        print("\n\t model saved!!!")
        minLoss=totLoss #float(results[-1])

 counter= 0
 counter= 100
 counter= 200
 counter= 300
 counter= 400
 counter= 500
 counter= 600
 counter= 700

	 epoch: 0 	minLoss: 100000.0  totLoss: 75057.38637542725

	 model saved!!!
 counter= 0
 counter= 100
 counter= 200
 counter= 300
 counter= 400
 counter= 500
 counter= 600
 counter= 700

	 epoch: 1 	minLoss: 75057.38637542725  totLoss: 53581.09680938721

	 model saved!!!
 counter= 0
 counter= 100
 counter= 200
 counter= 300
 counter= 400
 counter= 500
 counter= 600
 counter= 700

	 epoch: 2 	minLoss: 53581.09680938721  totLoss: 44025.64226770401

	 model saved!!!
 counter= 0
 counter= 100
 counter= 200
 counter= 300
 counter= 400
 counter= 500
 counter= 600
 counter= 700

	 epoch: 3 	minLoss: 44025.64226770401  totLoss: 37136.67612361908

	 model saved!!!
 counter= 0
 counter= 100
 counter= 200
 counter= 300
 counter= 400
 counter= 500
 counter= 600
 counter= 700

	 epoch: 4 	minLoss: 37136.67612361908  totLoss: 32015.7672829628

	 model saved!!!
 counter= 0
 counter= 100
 cou